<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/master/Fal_Detection_System_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [ ]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip

# Implementation

In [ ]:
class AddressProcessor:

  def __init__(self, 
               data_path = "SisFall_dataset/")
  self.data_path = data_path
  
  def get_file_name(self):
  allfiles = []
  allFolders = glob.glob(self.data_path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

  def split_address(self):
  dataset_address = self.get_file_name()
  np.random.shuffle(dataset_address)
  self.train, self.test = np.split(dataset_address, [int(len(dataset_address)*0.7)])

  def main(self):
    self.split_address()


Get all addresses

In [ ]:
def get_file_name(path):
  allfiles = []
  allFolders = glob.glob(path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

In [ ]:
get_file_name("SisFall_dataset/")

array(['SisFall_dataset/SA18/F11_SA18_R03.txt',
       'SisFall_dataset/SA18/F03_SA18_R05.txt',
       'SisFall_dataset/SA18/F02_SA18_R04.txt', ...,
       'SisFall_dataset/SE07/D14_SE07_R05.txt',
       'SisFall_dataset/SE07/D16_SE07_R05.txt',
       'SisFall_dataset/SE07/D12_SE07_R04.txt'], dtype='<U37')

read dataset from address path

In [ ]:
class DatasetProcessor:
  
  def __init__(self, 
               train_dataset,
               test_dataset,
               window_size,
               down_sampleing_factor)
  self.train_dataset = train_dataset
  self.test_dataset = test_dataset
  self.window_size = window_size
  self.down_sampleing_factor = down_sampleing_factor
  
  def read_data(self, data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

  def add_features(self, data_path):
    dataset = self.read_data(data_path)
    new_dataset = pd.DataFrame()
    new_dataset['acc_1'] = dataset.apply(
        lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
    new_dataset['acc_2'] = dataset.apply(
        lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
    new_dataset['geo'] = dataset.apply(
        lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
    new_dataset['label'] = get_label(data_path)
    return np.round(new_dataset.to_numpy(),2)

  def get_label(self, data_path):
    label = data_path[21]
    if label =='D':
      return int(0)
    elif label =='F':  
      label_path = data_path.replace('dataset','enhanced')
      labels = pd.read_csv(label_path,header=None)
      labels[labels == 2] = 1
      return labels

  def datasets_to_nparray(self):
    result = np.empty((0, 4), int)
    for address in datasets_address_array:
      result = np.concatenate(
          (result,self.add_features(read_data(address),address)),axis=0)
    return result
  
  def windowing(self, dataset):
    window = self.window_size * (dataset.shape[1]-1)
    cut = dataset.shape[0] % self.window_size
    feature = dataset[:-cut,0:-1]
    label = dataset[:-cut,-1]
    feature = feature.ravel().reshape(feature.size//window,window)
    label = label.reshape(label.size//self.window_size,self.window_size)
    label = label.sum(axis=1)
    label[label > 0] = 1
    return feature,label

  def dataset_to_tensor(self):
    test_feature, test_label = self.windowing(self.datasets_to_nparray(self.test_dataset),self.window_size)
    np.savez('Sisfall_data_test', inputs=test_feature, targets=test_label)
    train_feature, train_label = self.windowing(self.datasets_to_nparray(self.test_dataset),self.window_size)
    np.savez('Sisfall_data_train', inputs=train_feature, targets=train_label)

  def dataset_loader(self):
    npz = np.load("Sisfall_data_train.npz")
    self.train_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
    self.train_targets = npz["targets"].astype(np.int)
    
    npz = np.load("Sisfall_data_test.npz")
    self.test_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
    self.test_targets = npz["targets"].astype(np.int)

  def downsampling(self):
      positive = self.train_inputs[self.train_inputs['targets']==1]
      negative = self.train_inputs[self.train_inputs['targets']==0].sample(n=int(len(positive)*self.down_sampleing_factor))
      return pd.concat([positive, negative], ignore_index=True).sample(frac=1).reset_index(drop=True)

  def main(self):
    self.dataset_to_tensor()
    self.dataset_loader()
    self.downsampling()


# Main

Data Pre-processing

Neural Network Training

In [ ]:
class Models:

  def __init__(self, 
               X_train,
               y_train,
               X_test,
               y_test,
               batch_size = 128,
               n_epochs = 100,
               filters = 250,               
               kernel_size = 3,
               units = 250,
               optimizer = tf.optimizers.SGD,
               drop_rate = 0.4,
               learning_rate = 0.02,
               validation_split = 0.10
               ):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.filters = filters
        self.kernel_size = kernel_size
        self.units = units
        self.optimizer = optimizer
        self.drop_rate = drop_rate
        self.learning_rate = learning_rate
        self.validation_split = validation_split

  def define_mlp(self):

    self.mlp_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(self.input_size,activation="relu"),
                                tf.keras.layers.Dense(self.hidden_layer_size, activation="relu"),
                                tf.keras.layers.Dense(self.hidden_layer_size,activation="relu"),
                                tf.keras.layers.Dense(self.output_size, activation='sigmoid')
                                ])

    self.mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  def train_mlp(self):
    self.mlp_history = self.mlp_model.fit(
        self.X_train,
        self.y_train,
        batch_size = self.batch_size,
        epochs = self.n_epochs,
        shuffle = True,
        verbose = 1
          )

  def plot_cnn_training_history(self):
    return plot_history(self.mlp_history)

  def evaluate_mlp(self):
    self.mlp_prediction = self.mlp_model.predict(self.X_test, verbose = 1, batch_size = self.batch_size)
    print(classification_report(self.y_test, self.mlp_prediction))
    print(confusion_matrix(self.y_test, self.mlp_prediction))
    

  def define_lstm(self):
    return
  
  def train_lstm(self):
    return

  def evaluate_lstm(self):
    return
    
  def define_svm(self):
    self.svm_model = LinearSVC(C = 0.0001)
    self.svm_model.fit(self.X_train, self.y_train)

  def evaluate_svm(self):
    self.svm_prediction = self.svm_model.predict(self.X_test)
    print(classification_report(self.y_test, self.svm_prediction))
    print(confusion_matrix(self.y_test, self.svm_prediction))
  
    def define_LGR(self):
    self.LGR_model = LogisticRegression()
    self.LGR_model.fit(self.X_train, self.y_train)
    
  def evaluate_LGR(self):
    self.LGR_prediction = self.LGR_model.predict(self.X_test)
    print(classification_report(self.y_test, self.LGR_prediction))
    print(confusion_matrix(self.y_test, self.LGR_prediction))
  
  def define_knn(self):
    self.knn_model = KNeighborsClassifier(n_neighbors=4)
    self.knn_model.fit(self.X_train, self.y_train)
    
  def evaluate_knn(self):
    self.knn_prediction = self.knn_model.predict(self.X_test)
    print(classification_report(self.y_test, self.knn_prediction))
    print(confusion_matrix(self.y_test, self.knn_prediction))

  def define_ensemble_concept(self):

  def evaluate_ensemble_concept(self):

Prediction based on Ensemble Concept

In [ ]:
def ensmble_concept(Knn_prediction,model_prediction):
  return np.logical_or(Knn_prediction , model_prediction.T.ravel().round())

result = ensmble_concept(y_pred,predictions)
print(classification_report(y_test, result))
print(confusion_matrix(y_test, result))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     22857
           1       0.65      0.81      0.72      1072

    accuracy                           0.97     23929
   macro avg       0.82      0.90      0.85     23929
weighted avg       0.98      0.97      0.97     23929

[[22379   478]
 [  201   871]]


Models' Summary

In [ ]:
conclusion = pd.DataFrame([['Neural Network',precision_1,recall_1,f1Score_1],
              ['Logistic Regression',0.53,0.17,0.26],
              ['SVM',0.38,0.01,0.03],
              ['KNN',0.94,0.56,0.70],
              ['Neural Network after Balancing',precision_2,recall_2,f1Score_2],
              ['Logistic Regression after Balancing',0.08,0.93,0.15],
              ['SVM after Balancing',0.06,0.97,0.11],
              ['KNN after Balancing',0.73,0.73,0.73],
              ['Ensemble concept after Balancing',0.65,0.81,0.72]],
              columns=["Algorithm","Precision","Recall","F1score"])
conclusion = conclusion.set_index('Algorithm')
conclusion.style.background_gradient(cmap="YlOrRd")


,Precision,Recall,F1score
Algorithm,,,
Neural Network,0.990000,0.340000,0.510000
Logistic Regression,0.530000,0.170000,0.260000
SVM,0.380000,0.010000,0.030000
KNN,0.940000,0.560000,0.700000
Neural Network after Balancing,0.880000,0.580000,0.700000
Logistic Regression after Balancing,0.080000,0.930000,0.150000
SVM after Balancing,0.060000,0.970000,0.110000
KNN after Balancing,0.730000,0.730000,0.730000
Ensemble concept after Balancing,0.650000,0.810000,0.720000


As you see After Blancing the dataset Ensemble concept and KNN model do the best !